# Kitty Classifier Training

Dataset credits:

### Import all the Dependencies

In [1]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt
from IPython.display import HTML
import scipy
import numpy as np

2025-01-15 21:48:16.022646: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-15 21:48:16.022720: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-15 21:48:16.064177: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-15 21:48:16.150731: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-15 21:48:16.996122: W tensorflow/compiler/tf2

### Verify TensorFlow GPU Support

In [2]:
tf.config.list_physical_devices("GPU")

2025-01-15 21:48:20.481551: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_COMPAT_NOT_SUPPORTED_ON_DEVICE: forward compatibility was attempted on non supported HW
2025-01-15 21:48:20.481579: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:129] retrieving CUDA diagnostic information for host: oles-linux
2025-01-15 21:48:20.481586: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:136] hostname: oles-linux
2025-01-15 21:48:20.481643: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:159] libcuda reported version is: 535.183.1
2025-01-15 21:48:20.481662: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:163] kernel reported version is: 535.171.4
2025-01-15 21:48:20.481668: E external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:244] kernel version 535.171.4 does not match DSO version 535.183.1 -- cannot find working devices in this configuration


[]

In [3]:
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

GPU is NOT AVAILABLE


In [4]:
IMAGE_SIZE = 256
TARGET_SIZE = (256, 256)
CHANNELS = 3

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=5,  # Reduce rotation range
    horizontal_flip=True,
    preprocessing_function=lambda x: tf.image.resize(x, TARGET_SIZE)
)
train_generator = train_datagen.flow_from_directory(
    "dataset/train",
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=32,
    class_mode="sparse"
)

validation_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=5,  # Reduce rotation range
    horizontal_flip=True,
    preprocessing_function=lambda x: tf.image.resize(x, TARGET_SIZE)
)
validation_generator = validation_datagen.flow_from_directory(
    'dataset/val',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=32,
    class_mode="sparse",
    shuffle=True
)

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_SIZE, IMAGE_SIZE, CHANNELS)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(len(class_names), activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=20,
    validation_data=validation_generator,
    validation_steps=len(validation_generator)
)


NameError: name 'ImageDataGenerator' is not defined

### Import data into tensorflow dataset object

In [4]:
IMAGE_SIZE = 256
TARGET_SIZE = (256, 256)
CHANNELS = 3

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=10,
        horizontal_flip=True,
        preprocessing_function=lambda x: tf.image.resize(x, TARGET_SIZE)
)
train_generator = train_datagen.flow_from_directory(
        "dataset/train",
        target_size=(IMAGE_SIZE,IMAGE_SIZE),
        batch_size=32,
        class_mode="sparse"
)

Found 4880 images belonging to 20 classes.


In [8]:
train_generator.class_indices

{'Abyssinian': 0,
 'American Bobtail': 1,
 'American Curl': 2,
 'American Shorthair': 3,
 'Bengal': 4,
 'Birman': 5,
 'Bombay': 6,
 'British Shorthair': 7,
 'Egyptian Mau': 8,
 'Exotic Shorthair': 9,
 'Maine Coon': 10,
 'Manx': 11,
 'Norwegian Forest': 12,
 'Persian': 13,
 'Ragdoll': 14,
 'Russian Blue': 15,
 'Scottish Fold': 16,
 'Siamese': 17,
 'Sphynx': 18,
 'Turkish Angora': 19}

In [7]:
class_names = list(train_generator.class_indices.keys())
class_names

['Abyssinian',
 'American Bobtail',
 'American Curl',
 'American Shorthair',
 'Bengal',
 'Birman',
 'Bombay',
 'British Shorthair',
 'Egyptian Mau',
 'Exotic Shorthair',
 'Maine Coon',
 'Manx',
 'Norwegian Forest',
 'Persian',
 'Ragdoll',
 'Russian Blue',
 'Scottish Fold',
 'Siamese',
 'Sphynx',
 'Turkish Angora']

In [6]:
validation_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=10,
        horizontal_flip=True,
        preprocessing_function=lambda x: tf.image.resize(x, TARGET_SIZE)
)
validation_generator = validation_datagen.flow_from_directory(
        'dataset/val',
        target_size=(IMAGE_SIZE,IMAGE_SIZE),
        batch_size=32,
        class_mode="sparse",
        shuffle=True
)

Found 700 images belonging to 20 classes.


In [9]:
test_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    horizontal_flip=True,
    preprocessing_function=lambda x: tf.image.resize(x, TARGET_SIZE)
)

test_generator = test_datagen.flow_from_directory(
        'dataset/test',
        target_size=(IMAGE_SIZE,IMAGE_SIZE),
        batch_size=32,
        class_mode="sparse",
        shuffle=True
)

Found 1420 images belonging to 20 classes.


## Building the Model

In [15]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_SIZE, IMAGE_SIZE, CHANNELS)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(len(class_names), activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

2024-02-21 23:44:34.934656: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-21 23:44:34.935017: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-21 23:44:34.935232: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_16 (Conv2D)          (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d_16 (MaxPooli  (None, 127, 127, 32)      0         
 ng2D)                                                           
                                                                 
 conv2d_17 (Conv2D)          (None, 125, 125, 64)      18496     
                                                                 
 max_pooling2d_17 (MaxPooli  (None, 62, 62, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_18 (Conv2D)          (None, 60, 60, 128)       73856     
                                                                 
 max_pooling2d_18 (MaxPooli  (None, 30, 30, 128)       0

### Compiling the Model
We use `adam` Optimizer, `SparseCategoricalCrossentropy` for losses, `accuracy` as a metric

In [12]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

### Figure out how many steps per epoch:

Training dataset has 4901 images

In [13]:
4901/32

153.15625

Validation dataset has 721 images

In [14]:
721/32

22.53125

In [17]:
tf.debugging.set_log_device_placement(True)

In [18]:
#New training test
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=20,
    validation_data=validation_generator,
    validation_steps=len(validation_generator)
)

Epoch 1/20


2024-02-21 23:45:22.816973: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-02-21 23:45:25.465918: I external/local_xla/xla/service/service.cc:168] XLA service 0x7fe78c028160 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-02-21 23:45:25.465942: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 970, Compute Capability 5.2
2024-02-21 23:45:25.477014: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1708577125.573124   29421 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


153/153 [==============================] - 79s 475ms/step - loss: 3.0062 - accuracy: 0.0684 - val_loss: 2.8722 - val_accuracy: 0.0871
Epoch 2/20
153/153 [==============================] - 68s 444ms/step - loss: 2.8871 - accuracy: 0.1086 - val_loss: 2.8398 - val_accuracy: 0.1371
Epoch 3/20
153/153 [==============================] - 68s 446ms/step - loss: 2.8138 - accuracy: 0.1389 - val_loss: 2.7445 - val_accuracy: 0.1671
Epoch 4/20
153/153 [==============================] - 69s 452ms/step - loss: 2.7114 - accuracy: 0.1658 - val_loss: 2.6402 - val_accuracy: 0.1843
Epoch 5/20
153/153 [==============================] - 71s 465ms/step - loss: 2.6029 - accuracy: 0.2074 - val_loss: 2.5963 - val_accuracy: 0.1871
Epoch 6/20
153/153 [==============================] - 70s 457ms/step - loss: 2.4326 - accuracy: 0.2553 - val_loss: 2.5022 - val_accuracy: 0.2286
Epoch 7/20
153/153 [==============================] - 69s 448ms/step - loss: 2.2817 - accuracy: 0.2898 - val_loss: 2.4609 - val_accuracy: 0.2

In [19]:
history = model.fit(
    train_generator,
    steps_per_epoch=153,
    batch_size=32,
    validation_data=validation_generator,
    validation_steps=23,
    verbose=1,
    epochs=20,
)

Epoch 1/20
153/153 [==============================] - 57s 371ms/step - loss: 2.5903 - accuracy: 0.2016
Epoch 2/20
  7/153 [>.............................] - ETA: 53s - loss: 2.5391 - accuracy: 0.2188

KeyboardInterrupt: 

In [ ]:
scores = model.evaluate(test_generator)

: 

In [ ]:
scores

: 

Scores is just a list containing loss and accuracy value

### Plotting the Accuracy and Loss Curves

In [ ]:
history

: 

You can read documentation on history object here: https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/History

In [ ]:
history.params

: 

In [ ]:
history.history.keys()

: 

**loss, accuracy, val loss etc are a python list containing values of loss, accuracy etc at the end of each epoch**

In [ ]:
type(history.history['loss'])

: 

In [ ]:
len(history.history['loss'])

: 

In [ ]:
history.history['loss'][:5] # show loss for first 5 epochs

: 

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

: 

In [ ]:
val_acc

: 

In [ ]:
acc

: 

In [ ]:
# Plotting the Accuracy and Loss Curves
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

EPOCHS = 20

plt.figure(figsize=(12, 6))

# Plotting Training and Validation Accuracy
plt.subplot(1, 2, 1)
plt.plot(range(1, EPOCHS + 1), acc, label='Training Accuracy')
plt.plot(range(1, EPOCHS + 1), val_acc, label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Plotting Training and Validation Loss
plt.subplot(1, 2, 2)
plt.plot(range(1, EPOCHS + 1), loss, label='Training Loss')
plt.plot(range(1, EPOCHS + 1), val_loss, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

: 

### Run prediction on a sample image

In [ ]:
import numpy as np


for image_batch, label_batch in test_generator:
    first_image = image_batch[0]
    first_label = int(labels_batch[0])
    
    print("first image to predict")
    plt.imshow(first_image)
    print("actual label:",class_names[first_label])
    
    batch_prediction = model.predict(images_batch)
    print("predicted label:",class_names[np.argmax(batch_prediction[0])])
    
    break

: 

### Write a function for inference

In [ ]:
def predict(model, img):
    img_array = tf.keras.preprocessing.image.img_to_array(images[i])
    img_array = tf.expand_dims(img_array, 0)

    predictions = model.predict(img_array)

    predicted_class = class_names[np.argmax(predictions[0])]
    confidence = round(100 * (np.max(predictions[0])), 2)
    return predicted_class, confidence

: 

**Now run inference on few sample images**

In [ ]:
plt.figure(figsize=(15, 15))
for images, labels in test_generator:
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i])
        
        predicted_class, confidence = predict(model, images[i])
        actual_class = class_names[int(labels[i])] 
        
        plt.title(f"Actual: {actual_class},\n Predicted: {predicted_class}.\n Confidence: {confidence}%")
        
        plt.axis("off")
    break

: 

### Saving the Model

Save model in h5 format so that there is just one file and we can upload that to GCP conveniently

In [21]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler
import numpy as np

# Define constants
IMAGE_SIZE = 256
BATCH_SIZE = 32
EPOCHS = 50
NUM_CLASSES = len(class_names)
LEARNING_RATE = 0.001

# Define data augmentation parameters
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    preprocessing_function=lambda x: tf.image.resize(x, (IMAGE_SIZE, IMAGE_SIZE))
)

train_generator = train_datagen.flow_from_directory(
    "dataset/train",
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='sparse'
)

validation_datagen = ImageDataGenerator(
    rescale=1./255,
    preprocessing_function=lambda x: tf.image.resize(x, (IMAGE_SIZE, IMAGE_SIZE))
)

validation_generator = validation_datagen.flow_from_directory(
    'dataset/val',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='sparse'
)

# Define learning rate schedule
def lr_schedule(epoch):
    if epoch < 10:
        return LEARNING_RATE
    elif epoch < 30:
        return LEARNING_RATE * 0.1
    else:
        return LEARNING_RATE * 0.01

lr_scheduler = LearningRateScheduler(lr_schedule)

# Define model architecture
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(256, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(NUM_CLASSES, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    callbacks=[lr_scheduler]
)

# Evaluate the model on test data
test_datagen = ImageDataGenerator(
    rescale=1./255,
    preprocessing_function=lambda x: tf.image.resize(x, (IMAGE_SIZE, IMAGE_SIZE))
)

test_generator = test_datagen.flow_from_directory(
    'dataset/test',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='sparse'
)

scores = model.evaluate(test_generator)
print("Test Loss:", scores[0])
print("Test Accuracy:", scores[1])


Found 4880 images belonging to 20 classes.
Found 700 images belonging to 20 classes.
Epoch 1/50
153/153 [==============================] - 72s 462ms/step - loss: 3.0043 - accuracy: 0.0725 - val_loss: 2.9011 - val_accuracy: 0.1000 - lr: 0.0010
Epoch 2/50
153/153 [==============================] - 71s 462ms/step - loss: 2.9034 - accuracy: 0.1008 - val_loss: 2.8423 - val_accuracy: 0.1043 - lr: 0.0010
Epoch 3/50
153/153 [==============================] - 70s 456ms/step - loss: 2.8586 - accuracy: 0.1227 - val_loss: 2.8065 - val_accuracy: 0.1400 - lr: 0.0010
Epoch 4/50
153/153 [==============================] - 69s 451ms/step - loss: 2.8292 - accuracy: 0.1289 - val_loss: 2.7597 - val_accuracy: 0.1500 - lr: 0.0010
Epoch 5/50
153/153 [==============================] - 70s 460ms/step - loss: 2.7826 - accuracy: 0.1451 - val_loss: 2.7302 - val_accuracy: 0.1657 - lr: 0.0010
Epoch 6/50
153/153 [==============================] - 68s 443ms/step - loss: 2.7328 - accuracy: 0.1607 - val_loss: 2.6743 - v

In [ ]:
model.save("../cats.h5")

: 

: 

: 